In [1]:
'''
Author: Alemu Sisay Nigru
Matricola Number: 730159
'''

In [ ]:
#!pip install keras-tuner
!pip install torchsummary

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from tqdm import tqdm
import cv2
from PIL import Image
from torchsummary import summary
from sklearn.model_selection import train_test_split
from torchvision import models,transforms
#base_skin_dir = os.path.join('..', 'input')


In [3]:
#from keras_tuner import RandomSearch
#from keras_tuner.engine.hyperparameters import HyperParameters
import torch
from torch import optim,nn
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
from torchvision import models,transforms
import tensorflow as tf
from tensorflow import keras

In [4]:
import os
paths = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))
df_all = pd.read_csv('../input/ground-truth/HAM10000_metadata (1).csv')
df_all.head()

In [5]:
#from google.colab import drive
#drive.mount("/content/drive", force_remount=True)
ground_truth = '../input/ground-truth/HAM10000_metadata (1).csv'
base_skin_dir = "../input/skin-cancer-mnist-ham10000"

In [6]:
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}
print(len(imageid_path_dict))
lesion_type_dict = {
    'nv': 'Melanocytic nevus',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [7]:
df_all = pd.read_csv(ground_truth)
df_all['path'] = df_all['image_id'].map(imageid_path_dict.get)
df_all['cell_type'] = df_all['dx'].map(lesion_type_dict.get) 
df_all['cell_type_idx'] = pd.Categorical(df_all['cell_type']).codes
df_all.head()

In [8]:
df_all.describe(exclude=[np.number])

In [9]:
df_df = df_all[df_all['dx']=='df']
df_vasc = df_all[df_all['dx']=='vasc']
df_akiec = df_all[df_all['dx']=='akiec']
df_bcc = df_all[df_all['dx']=='bcc']
df_bkl = df_all[df_all['dx']=='bkl']
df_mel = df_all[df_all['dx']=='mel']
df_nv = df_all[df_all['dx']=='nv'][:1690]
df_all = pd.concat([df_df,df_vasc,df_akiec,df_bcc,df_bkl,df_mel,df_nv],ignore_index=True)
len(df_all)

In [10]:
df_all['dx'].value_counts()

In [11]:
fig, ax1 = plt.subplots(1, 1, figsize = (10, 5))
df_all['dx'].value_counts().plot(kind='bar', ax=ax1)
df_all['cell_type'].value_counts()

In [12]:
def compute_img_mean_std(image_paths):
    """
        computing the mean and std of three channel on the whole dataset,
        first we should normalize the image from 0-255 to 0-1
    """

    img_h, img_w = 128, 128
    imgs = []
    means, stdevs = [], []

    for i in tqdm(range(len(image_paths))):
        img = cv2.imread(image_paths[i])
        img = cv2.resize(img, (img_h, img_w))
        imgs.append(img)

    imgs = np.stack(imgs, axis=3)
    print(imgs.shape)

    imgs = imgs.astype(np.float32) / 255.

    for i in range(3):
        pixels = imgs[:, :, i, :].ravel()  # resize to one row
        means.append(np.mean(pixels))
        stdevs.append(np.std(pixels))

    means.reverse()  # BGR --> RGB
    stdevs.reverse()

    print("normMean = {}".format(means))
    print("normStd = {}".format(stdevs))
    return means,stdevs

In [13]:
all_image_path = df_all['path']
norm_mean,norm_std = compute_img_mean_std(all_image_path)

In [14]:
# now we create a val set using df because we are sure that none of these images have augmented duplicates in the train set
y = df_all['cell_type_idx']
_, df_temp = train_test_split(df_all, test_size=0.3, random_state=101, stratify=y)
label = df_temp['cell_type_idx']
df_val, df_test= train_test_split(df_temp, test_size=0.5, random_state=25, stratify=label)
df_test.shape

In [15]:
df_val['cell_type_idx'].value_counts()

In [16]:
# This set will be df_original excluding all rows that are in the val set
# This function identifies if an image is part of the train or val set.
def get_val_rows(x):
    # create a list of all the lesion_id's in the val set
    val_list = list(df_val['image_id'])
    test_list = list(df_test['image_id'])
    if str(x) in val_list:
        return 'val'
    elif str(x) in test_list:
        return 'test'
    else:
        return 'train'

# identify train and val rows
# create a new colum that is a copy of the image_id column
df_all['train_val_or_test'] = df_all['image_id']
# apply the function to this new column
df_all['train_val_or_test'] = df_all['train_val_or_test'].apply(get_val_rows)
# filter out train rows
df_train = df_all[df_all['train_val_or_test'] == 'train']
print(len(df_train))
print(len(df_val))
print(len(df_test))

In [17]:
plt.figure(figsize=(12, 4))
df_train['dx'].value_counts().plot(x = df_train['dx'].value_counts(), kind='bar', color=["green","yellow", "red",  "blue", "purple","black","cyan"],title ='Training data distribution')
# new helper method to auto-label bars

In [18]:
df_val=df_val.reset_index(drop=True)
df_test=df_test.reset_index(drop=True)

In [19]:
weights = np.ceil(max(df_train['dx'].value_counts())/df_train['cell_type_idx'].value_counts())
weights

In [20]:
#Duplicate some columns to make them balanced 
# Duplicate fewer class to balance through all classes
data_aug_rate = [6,4,2,15,1,2,12]
for i in range(7):
    if data_aug_rate[i]:
        df_train=df_train.append([df_train.loc[df_train['cell_type_idx'] == i,:]]*(data_aug_rate[i]-1), ignore_index=True)
df_train['dx'].value_counts()

In [21]:
df_train['dx'].value_counts().plot(x = df_train['dx'].value_counts(), kind='bar', color=["green","yellow", "red",  "blue", "purple","black","cyan"],title ='Training data distribution')

In [22]:
#normMean = [0.7611847, 0.5426958, 0.56694466]
#normStd = [0.13939652, 0.14886442, 0.1642639]
#Define the transformation of the train images.
input_size = 128
train_transform = transforms.Compose([transforms.Resize((input_size,input_size)),transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),transforms.RandomRotation(20),
                                      transforms.ColorJitter(brightness=0.1, contrast=0.1, hue=0.1),
                                      transforms.ToTensor(), transforms.Normalize(norm_mean, norm_std)])
# define the transformation of the val images.
val_transform = transforms.Compose([transforms.Resize((input_size,input_size)), transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])
# define the transformation of the test images.
test_transform = transforms.Compose([transforms.Resize((input_size,input_size)), transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])

In [23]:
# Define a pytorch dataloader for this dataset
class HAM5000(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Load data and get label
        X = Image.open(self.df['path'][index])
        y = torch.tensor(int(self.df['cell_type_idx'][index]))

        if self.transform:
            X = self.transform(X)

        return X, y

In [24]:
# Define the training set using the table train_df and using our defined transitions (train_transform)
training_set = HAM5000(df_train, transform=train_transform)
train_loader = DataLoader(training_set, batch_size=128, shuffle=True)
#Doing the same for the validation set:
validation_set = HAM5000(df_val, transform=val_transform)
val_loader = DataLoader(validation_set, batch_size=128, shuffle=False)
# And for the test set:
test_set = HAM5000(df_test, transform=test_transform)
test_loader = DataLoader(test_set, batch_size=len(df_test), shuffle=False)

In [25]:
# Implementation of CNN/ConvNet Model
class CNN1(torch.nn.Module):

    def __init__(self):
        super(CNN1, self).__init__()

        self.feature_extractor = nn.Sequential(
            # L1 ImgIn shape=(?, 128, 128, 3)
            # Conv -> (?, 126, 126, 16)
            # Pool -> (?, 63, 63, 16)            
            nn.Conv2d(3, 16, 3), #conv1
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            #nn.Dropout(p=0.2),
            
            # L2 ImgIn shape=(?, 63, 63, 16)
            # Conv      ->(?, 61, 61, 32)
            # Pool      ->(?, 30, 30, 32)    
            nn.Conv2d(16, 32, 3), #conv2
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2,stride =2),
            nn.ReLU(),
            #nn.Dropout(p=0.2),
            ################################
            # L3 ImgIn shape=(?, 30, 30, 32)
            # Conv ->(?, 28, 28, 64)  
            #   
            nn.Conv2d(32, 64, 3), #conv3
            nn.BatchNorm2d(64),
            nn.ReLU(),
            #nn.Dropout(p=0.2),
            #L4 ImgIn shape=(?, 28, 28, 64) 
            # Conv ->(?, 26, 26, 64)  
            # Pool ->(?, 13, 13, 64)  
            nn.Conv2d(64, 64, 3), #conv4
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2,stride =2),
            nn.ReLU(),
            #nn.Dropout(p=0.2),
            ####################################   

        )    

        # L7 FC 28x28x128 inputs -> 2048 outputs 
        self.fc1 = nn.Linear(13 * 13 * 64, 140)
        torch.nn.init.xavier_uniform_(self.fc1.weight)   
        self.layer4 = nn.Sequential(
            self.fc1,
            nn.ReLU(),
            nn.Dropout(p=0.2))
        
        self.fc2 = nn.Linear(140, 32)
        self.layer5 = nn.Sequential(self.fc2,nn.ReLU(),nn.Dropout(p=0.2))
        # L8 Final FC 50 inputs -> 7 outputs
        self.fc3 = nn.Linear(32, 7)
        torch.nn.init.xavier_uniform_(self.fc3.weight) # initialize parameters        

    def forward(self, x):
        conv_features = self.feature_extractor(x)
        #print(conv_features.shape)
        conv_features = conv_features.view(x.shape[0], -1)
        #out = out.view(out.size(0), -1)   
        out = self.fc1(conv_features)
        out = self.fc2(out)
        out = self.fc3(out)
        return out


#instantiate CNN model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1= CNN1().to(device)
summary(model1,input_size=(3,128,128))

In [27]:
# we use Adam optimizer, use cross entropy loss as our loss function
from torch.optim.lr_scheduler import ReduceLROnPlateau 
optimizer = optim.Adam(model1.parameters(), lr=5e-4, weight_decay=4e-3)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)

In [28]:
# this function is used during training process, to calculation the loss and accuracy
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [29]:
total_loss_train, total_acc_train = [],[]
def train(train_loader,device, model, criterion, optimizer, epoch, scheduler):
    model.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    curr_iter = (epoch - 1) * len(train_loader)
    # initialize the early_stopping object
    #early_stopping = EarlyStopping(patience=patience, verbose=True)
    for i, data in enumerate(train_loader):
        images, labels = data
        N = images.size(0)
        # print('image shape:',images.size(0), 'label shape',labels.size(0))
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        prediction = outputs.max(1, keepdim=True)[1]
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
        train_loss.update(loss.item())
        curr_iter += 1
        if (i + 1) % 25 == 0:
            print('[epoch %d], [iter %d / %d], [train loss %.5f], [train acc %.5f]' % (
                epoch, i + 1, len(train_loader), train_loss.avg, train_acc.avg))
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)
    return train_loss.avg, train_acc.avg

In [30]:
def validate(val_loader,device,  model, criterion, optimizer, epoch,scheduler):
    model.eval()
    val_loss = AverageMeter()
    val_acc = AverageMeter()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            images, labels = data
            N = images.size(0)
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            outputs = model(images)
            prediction = outputs.max(1, keepdim=True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)

            val_loss.update(criterion(outputs, labels).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss.avg, val_acc.avg))
    print('------------------------------------------------------------')
    return val_loss.avg, val_acc.avg

In [31]:
criterion = nn.CrossEntropyLoss().to(device)
print('Running on: ',device)

epoch_num = 50
best_val_acc = 0
total_loss_val, total_acc_val = [],[]
total_loss_tr, total_acc_tr = [],[]
for epoch in range(1, epoch_num+1):
    loss_train, acc_train = train(train_loader, device, model1, criterion, optimizer, epoch, scheduler)
    loss_val, acc_val = validate(val_loader, device, model1, criterion, optimizer, epoch, scheduler)
    total_loss_val.append(loss_val)
    total_acc_val.append(acc_val)
    total_loss_tr.append(loss_train)
    total_acc_tr.append(acc_train)    
    if acc_val > best_val_acc:
        best_val_acc = acc_val
        print('*****************************************************')
        print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, loss_val, acc_val))
        print('*****************************************************')
    scheduler.step(loss_val)


In [32]:
fig = plt.figure()
fig1 = fig.add_subplot(2,1,1)
fig2 = fig.add_subplot(2,1,2)
#fig1 = fig.add_subplot(2,1,1)
fig1.plot(total_loss_tr, label = 'Training loss')
fig1.plot(total_loss_val, label = 'Validation loss',linestyle='--')
fig2.plot(total_acc_tr, label = 'Training Accuracy')
fig2.plot(total_acc_val, label ='validation accuracy',linestyle='--')
plt.legend()
plt.show()

In [33]:
fig = plt.figure()

fig1 = fig.add_subplot(1,1,1)
#fig1 = fig.add_subplot(2,1,1)
fig1.plot(total_loss_tr, label = 'Training loss')
fig1.plot(total_loss_val, label = 'Validation loss',linestyle='--')
plt.legend()
plt.show()

In [34]:
fig = plt.figure()
fig2 = fig.add_subplot(1,1,1)
fig2.plot(total_acc_tr, label = 'Training Accuracy')
fig2.plot(total_acc_val, label ='validation accuracy',linestyle='--')
plt.legend()
plt.show()

In [35]:
def plot_confusion_matrix(cm, classes,
                      normalize=False,
                      title='Confusion matrix',
                      cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [36]:
def test(test_loader,device,  model, criterion):
    model.eval()
    test_loss = AverageMeter()
    test_acc = AverageMeter()
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            images, labels = data
            N = images.size(0)
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            outputs = model(images)
            prediction = outputs.max(1, keepdim=True)[1]

            test_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)

            test_loss.update(criterion(outputs, labels).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [test loss %.5f], [test acc %.5f]' % (1, test_loss.avg, test_acc.avg))
    print('------------------------------------------------------------')
    return test_loss.avg, test_acc.avg

In [37]:
loss_test, acc_test = test(test_loader, device, model1, criterion)

In [38]:
# sklearn libraries
import itertools
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
small_test = test_loader.dataset.df[:][0:300]
small_test_set = HAM5000(small_test, transform=test_transform)
small_test_loader = DataLoader(small_test_set, batch_size=len(small_test), shuffle=False)

In [39]:
model1.eval()
y_label = []
y_predict = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        images, labels = data
        N = images.size(0)
        images = Variable(images).to(device)
        outputs = model1(images)
        prediction = outputs.max(1, keepdim=True)[1]
        y_label.extend(labels.cpu().numpy())
        y_predict.extend(np.squeeze(prediction.cpu().numpy().T))

# compute the confusion matrix
confusion_mtx = confusion_matrix(y_label, y_predict)
# plot the confusion matrix
plot_labels = ['akiec', 'bcc', 'bkl', 'df', 'nv', 'vasc','mel']
plot_confusion_matrix(confusion_mtx, plot_labels)

In [40]:
# Generate a classification report
report = classification_report(y_label, y_predict, target_names=plot_labels)
print(report)

In [41]:
# Save
torch.save(model1.state_dict(),'Trained_CNN_Model1.pt')

In [ ]:
model1.load_state_dict(torch.load('Trained_CNN_Model1.pt'))


In [42]:
#Testing on a random image in the dataset
test_transform = transforms.Compose([transforms.Resize((input_size,input_size)), transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])
random_test_set = HAM5000(df_test[0:5], transform=test_transform)
random_test_loader = DataLoader(random_test_set, batch_size=len(random_test_set), shuffle=False) 
with torch.no_grad():
    for i, data in enumerate(random_test_loader):
        images, labels = data
        N = images.size(0)
        images = Variable(images).to(device)
        outputs = model1(images)
        prediction = outputs.max(1, keepdim=True)[1]
        y_label.extend(labels.cpu().numpy())
        y_predict.extend(np.squeeze(prediction.cpu().numpy().T))
        
print('Prediction:',y_predict[:5],'\nTrue label:',y_label[:5])     

In [43]:
from sklearn.model_selection import KFold

from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
torch.manual_seed(42)

criterion = nn.CrossEntropyLoss().to(device)

dataset = ConcatDataset([training_set, validation_set, test_set])

num_epochs=30
batch_size=128
k=4
splits=KFold(n_splits=k,shuffle=True,random_state=42)
foldperf={}

In [44]:
def train_epoch(model,device,dataloader,loss_fn,optimizer):
    train_loss,train_correct=0.0,0
    model.train()
    for images, labels in dataloader:

        images,labels = images.to(device),labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss = loss_fn(output,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        scores, predictions = torch.max(output.data, 1)
        train_correct += (predictions == labels).sum().item()

    return train_loss,train_correct
  
def valid_epoch(model,device,dataloader,loss_fn):
    valid_loss, val_correct = 0.0, 0
    model.eval()
    for images, labels in dataloader:
        images,labels = images.to(device),labels.to(device)
        output = model(images)
        loss=loss_fn(output,labels)
        valid_loss+=loss.item()*images.size(0)
        scores, predictions = torch.max(output.data,1)
        val_correct+=(predictions == labels).sum().item()

    return valid_loss,val_correct

In [45]:
LOSS = []
ACCURACY = []

VAL_LOSS = []
VAL_ACCURACY  = []
for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):

    print('Fold {}'.format(fold + 1))

    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    model = CNN1()
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.002)

    history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}

    for epoch in range(num_epochs):
        train_loss, train_correct=train_epoch(model,device,train_loader,criterion,optimizer)
        test_loss, test_correct=valid_epoch(model,device,test_loader,criterion)

        train_loss = train_loss / len(train_loader.sampler)
        train_acc = train_correct / len(train_loader.sampler) * 100
        test_loss = test_loss / len(test_loader.sampler)
        test_acc = test_correct / len(test_loader.sampler) * 100

        print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Test Loss:{:.3f} AVG Training Acc {:.2f} % AVG Test Acc {:.2f} %".format(epoch + 1,
                                                                                                             num_epochs,
                                                                                                             train_loss,
                                                                                                             test_loss,
                                                                                                             train_acc,
                                                                                                             test_acc))
        history['train_loss'].append(train_loss)
        history['test_loss'].append(test_loss)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)
        LOSS.append(train_loss)
        ACCURACY.append(train_acc)
        VAL_LOSS.append(test_loss)
        VAL_ACCURACY.append(test_acc)

    foldperf['fold{}'.format(fold+1)] = history  

torch.save(model,'k_cross_CNN.pt') 

In [58]:
x_axis = np.arange(1,num_epochs+1)
fig, ax = plt.subplots(4,2,constrained_layout=True, figsize=(13,16))

for i in range(4): 
    ax[i,0].set_title('Fold' + str(i+1) + ' Loss')
    ax[i,0].set_xlabel('Epoch')    
    ax[i,0].set_ylabel('Loss')
    ax[i,0].plot(x_axis, LOSS[num_epochs*i:num_epochs*(i+1)],label='Training loss')
    ax[i,0].plot(x_axis, VAL_LOSS[num_epochs*i:num_epochs*(i+1)],label='Validation loss', linestyle='--')
    ax[i,0].legend()
    
    ax[i,1].set_title('Fold' + str(i+1) + ' Accuracy')
    ax[i,1].set_ylabel('Accuracy')
    ax[i,1].plot(x_axis, ACCURACY[num_epochs*i:num_epochs*(i+1)]/(np.ones((num_epochs,))*100.0),label='Training accuracy')
    ax[i,1].plot(x_axis, VAL_ACCURACY[num_epochs*i:num_epochs*(i+1)]/(np.ones((num_epochs,))*100.0),label='Validation accuracy', linestyle='--')
    ax[i,1].set_xlabel('Epoch')
    ax[i,1].set_ylim((0,1))
    ax[i,1].set_xticks(np.arange(0,num_epochs+1,5))
    ax[i,1].legend()

In [61]:
loss = [LOSS[x:x+num_epochs] for x in range(0, len(LOSS), num_epochs)]
val_loss = [VAL_LOSS[x:x+num_epochs] for x in range(0, len(VAL_LOSS), num_epochs)]
acc = [ACCURACY[x:x+num_epochs] for x in range(0, len(ACCURACY), num_epochs)]
val_acc = [VAL_ACCURACY[x:x+num_epochs] for x in range(0, len(VAL_ACCURACY), num_epochs)]


In [62]:
loss = [sum(sub_list) / len(sub_list) for sub_list in zip(*loss)]
val_loss = [sum(sub_list) / len(sub_list) for sub_list in zip(*val_loss)]
acc = [sum(sub_list) / len(sub_list) for sub_list in zip(*acc)]
val_acc = [sum(sub_list) / len(sub_list) for sub_list in zip(*val_acc)]

In [64]:

x_axis = np.arange(1,num_epochs+1)
fig, ax = plt.subplots(1,2,constrained_layout=True, figsize=(12,5))

ax[0].set_title('Average Loss after 4 folds training')
ax[0].set_ylabel('Loss')
ax[0].set_xlabel('Epoch')
ax[0].plot(x_axis, loss,label='Training loss')
ax[0].plot(x_axis, val_loss,label='Validation loss', linestyle='--')
ax[0].legend()

ax[1].set_title('Average Accuracy after 4 folds training')
ax[1].set_ylabel('Accuracy')
ax[1].plot(x_axis, acc/(np.ones((num_epochs,))*100.0),label='Training accuracy')
ax[1].plot(x_axis, val_acc/(np.ones((num_epochs,))*100.0),label='Validation accuracy', linestyle='--')
ax[1].set_xlabel('Epoch')
ax[1].set_ylim((0,1))
ax[1].set_xticks(np.arange(0,num_epochs+1,5))
ax[1].legend()

In [ ]:
torch.load(model,'k_cross_CNN.pt') 

In [56]:
model.eval()
y_label = []
y_predict = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        images, labels = data
        N = images.size(0)
        images = Variable(images).to(device)
        outputs = model(images)
        prediction = outputs.max(1, keepdim=True)[1]
        y_label.extend(labels.cpu().numpy())
        y_predict.extend(np.squeeze(prediction.cpu().numpy().T))

# compute the confusion matrix
confusion_mtx = confusion_matrix(y_label, y_predict)
# plot the confusion matrix
plot_labels = ['akiec', 'bcc', 'bkl', 'df', 'nv', 'vasc','mel']
plot_confusion_matrix(confusion_mtx, plot_labels)

In [57]:
# Generate a classification report
report = classification_report(y_label, y_predict, target_names=plot_labels)
print(report)